# 自定义层

深度学习成功背后的一个因素是神经网络的灵活性：
我们可以用创造性的方式组合不同的层，从而设计出适用于各种任务的架构。
例如，研究人员发明了专门用于处理图像、文本、序列数据和执行动态规划的层。
有时我们会遇到或要自己发明一个现在在深度学习框架中还不存在的层。
在这些情况下，必须构建自定义层。本节将展示如何构建自定义层。

## 不带参数的层

首先，我们**构造一个没有任何参数的自定义层**。

回忆一下在`sec_model_construction`对块的介绍，
这应该看起来很眼熟。
下面的`CenteredLayer`类要从其输入中减去均值。
要构建它，我们只需继承基础层类并实现前向传播功能。

In [2]:
import torch
import torch.nn.functional as F
from torch import nn

class CenteredLayer(nn.Module):  # 和 model-construction 一样，继承 nn.Module
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()  # 不带参数的操作，类似于激活函数

让我们向该层提供一些数据，验证它是否能按预期工作。


In [3]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

现在，我们可以**将层作为组件合并到更复杂的模型中**。

In [4]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

作为额外的健全性检查，我们可以在向该网络发送随机数据后，检查均值是否为0。
由于我们处理的是浮点数，因为存储精度的原因，我们仍然可能会看到一个非常小的非零数。


In [5]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(3.7253e-09, grad_fn=<MeanBackward0>)

## 带参数的层

以上我们知道了如何定义简单的层，下面我们继续定义具有参数的层，
这些参数可以通过训练进行调整。

我们可以使用内置函数来创建参数，这些函数提供一些基本的管理功能。
比如管理访问、初始化、共享、保存和加载模型参数。
这样做的好处之一是：我们不需要为每个自定义层编写自定义的序列化程序。

现在，让我们实现自定义版本的全连接层。
回想一下，该层需要两个参数，**一个用于表示权重，另一个用于表示偏置项。**
在此实现中，我们使用修正线性单元作为激活函数。
该层需要输入参数：`in_units` 和 `units`，分别表示输入数和输出数。

In [20]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))  # 生成长度为 units 的行向量/一维张量！（注意不是列向量）（units 后加不加 逗号, 都可以）

    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data  # 矩阵与行向量的相加应用了广播机制
        return F.relu(linear)

备注：虽然在数学上我们常把偏置看作列向量（与矩阵乘法配合），但在 PyTorch 中，**self.bias.data 通常是一个一维张量，而不是二维矩阵或列向量。**（可理解为行向量）

它在加法运算时会自动通过广播机制扩展，去匹配与矩阵加法所需的形状。

接下来，我们实例化`MyLinear`类并访问其模型参数。


In [21]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[-0.5718,  0.7779,  0.5720],
        [ 0.5377,  1.5292, -0.7342],
        [-1.9317, -0.1345,  1.3219],
        [-0.2876,  1.2011, -0.8635],
        [-1.1341, -0.0282,  0.1273]], requires_grad=True)

我们可以使用自定义层直接执行前向传播计算。

In [22]:
linear(torch.rand(2, 5))

tensor([[0.0000, 1.4181, 0.4394],
        [0.0000, 0.0181, 0.0000]])

我们还可以使用自定义层构建模型，**就像使用内置的全连接层一样使用自定义层。**

In [23]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[2.6035],
        [0.4730]])

## 小结

* 我们可以通过基本层类设计自定义层。这允许我们定义灵活的新层，其行为与深度学习框架中的任何现有层不同。
* 在自定义层定义完成后，我们就可以在任意环境和网络架构中调用该自定义层。
* 层可以有局部参数，这些参数可以通过内置函数创建。

## 练习

1. 设计一个接受输入并计算张量降维的层，它返回$y_k = \sum_{i, j} W_{ijk} x_i x_j$。
2. 设计一个返回输入数据的傅立叶系数前半部分的层。

[Discussions](https://discuss.d2l.ai/t/1835)
